In [2]:
import requests
import re
import polars as pl
from bs4 import BeautifulSoup as bs

In [18]:
url = "https://www.ncbi.nlm.nih.gov/"

In [33]:
adicional = "protein/?term=caseine"

In [34]:
r = requests.get(url+adicional)
r
page = bs(r.text, 'html.parser')

In [35]:
#nombre
soup = page.find_all('p', {'class': 'title'})
lista = []
for i in range(len(soup)):
    nombre = soup[i].text
    lista.append(nombre)
df = pl.DataFrame({'Nombre': lista})
df

Nombre
str
"""CASEINE KINASE…"
"""caseine kinase…"
"""caseine kinase…"
"""caseine kinase…"
"""caseine kinase…"
"""caseine kinase…"
"""caseine kinase…"
"""caseine kinase…"
"""multicopy supr…"


In [36]:
#Accession
soup_2 = page.find_all('dl', {'class': 'rprtid'})
lista_2 = []
for nombres in range(len(soup_2)):
    nombre = soup_2[nombres].text
    nombre_regex = re.search(r'Accession:\s([A-Z0-9_.]*)', nombre)
    lista_2.append(nombre_regex.group(1))
df_2 = pl.DataFrame({'Accession':lista_2})
df_2

Accession
str
"""CAD26108.1"""
"""XP_003074047.1…"
"""XP_009265665.1…"
"""AFN84168.1"""
"""ADM12687.1"""
"""CDW72457.1"""
"""AGE94921.1"""
"""CAB05446.1"""
"""CAA70217.1"""


In [37]:
#GI
soup_3 = page.find_all('dl', {'class': 'rprtid'})
lista_3 = []
for gis in range(len(soup_3)):
    gi = soup_3[gis].text
    gi_regex = re.search(r'GI:\s([A-Z0-9_.]*)', gi)
    lista_3.append(gi_regex.group(1))
df_3 = pl.DataFrame({'GI':lista_3})
df_3

GI
str
"""19069723"""
"""303391635"""
"""685882473"""
"""396082559"""
"""303303196"""
"""678343414"""
"""449328644"""
"""1694914"""
"""1654007"""


In [38]:
#fasta

soup_4 = page.find_all('a', {'id': 'ReportShortCut6'})
lista_4 = []
for fastas in soup_4:
    url_2 = url+fastas['href']
    r_2 = requests.get(url_2)
    page_2 = bs(r_2.text, 'html.parser')
    soup_4 = page_2.find_all('div', {'class': 'seq gbff'})
    for escrito in range(len(soup_4)):
        fasta = soup_4[escrito].text
        lista_4.append(fasta)
df_4 = pl.DataFrame({'fasta': lista_4})
df_4

fasta
str
""""""
""""""
""""""
""""""
""""""
""""""
""""""
""""""
""""""


In [39]:
df_final = pl.concat([df, df_2, df_3, df_4], how="horizontal")
df_final

Nombre,Accession,GI,fasta
str,str,str,str
"""CASEINE KINASE…","""CAD26108.1""","""19069723""",""""""
"""caseine kinase…","""XP_003074047.1…","""303391635""",""""""
"""caseine kinase…","""XP_009265665.1…","""685882473""",""""""
"""caseine kinase…","""AFN84168.1""","""396082559""",""""""
"""caseine kinase…","""ADM12687.1""","""303303196""",""""""
"""caseine kinase…","""CDW72457.1""","""678343414""",""""""
"""caseine kinase…","""AGE94921.1""","""449328644""",""""""
"""caseine kinase…","""CAB05446.1""","""1694914""",""""""
"""multicopy supr…","""CAA70217.1""","""1654007""",""""""


In [57]:
df_final.write_csv('secuencias.csv')